In [8]:
from elasticsearch import Elasticsearch
import requests 
import tiktoken

In [9]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
client = Elasticsearch("http://localhost:9200/")

In [45]:
client.info()

ObjectApiResponse({'name': '1b11d203ef8a', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5MiujDNpR-2bO5ygOdS5zA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
index_name = "course-documents"

client.indices.create(index=index_name, body=index_settings)

In [12]:
for doc in documents:
    client.index(index=index_name, body=doc)

In [19]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            score = hit["_score"]
            title = hit["_source"]["question"]
            pretty_output = f"\nID: {id}\nQ: {title}\nScore: {score}"
            print(pretty_output)

In [43]:
query = "How do I execute a command in a running docker container?"
search = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}
r = client.search(index=index_name, body=search)
pretty_response(r)


ID: gbs5RZABv8A3YgVeM3Ju
Q: How do I debug a docker container?
Score: 84.050095

ID: j7s5RZABv8A3YgVeHXDD
Q: PGCLI - running in a Docker container
Score: 75.54128

ID: sLs5RZABv8A3YgVeQHMS
Q: Running multiple services in a Docker container
Score: 72.08518

ID: oLs5RZABv8A3YgVeNHKz
Q: How do I copy files from my local machine to docker container?
Score: 51.04628

ID: obs5RZABv8A3YgVeNHK9
Q: How do I copy files from a different folder into docker container’s working directory?
Score: 49.938507


In [46]:
search = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
r = client.search(index=index_name, body=search)
pretty_response(r)


ID: gbs5RZABv8A3YgVeM3Ju
Q: How do I debug a docker container?
Score: 84.050095

ID: oLs5RZABv8A3YgVeNHKz
Q: How do I copy files from my local machine to docker container?
Score: 51.04628

ID: obs5RZABv8A3YgVeNHK9
Q: How do I copy files from a different folder into docker container’s working directory?
Score: 49.938507


In [34]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [47]:
whole_context = ""
for hit in r["hits"]["hits"]:
    context = hit["_source"]
    whole_context += context_template.format(**context) + "\n\n"

prompt = prompt_template.format(question=query, context=whole_context.strip())
print(len(prompt))
print(prompt)

1462
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory containe

In [48]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

322